In [40]:
import os


In [41]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\NLP PROJECTS\\Text-Summarizer-project'

In [42]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 8E3C-0FAB

 Directory of c:\Users\Lenovo\Desktop\NLP PROJECTS\Text-Summarizer-project

30-03-2024  17:06    <DIR>          .
29-03-2024  16:14    <DIR>          ..
29-03-2024  17:12    <DIR>          .github
29-03-2024  16:09             3,238 .gitignore
30-03-2024  08:06    <DIR>          .pytest_cache
29-03-2024  17:14                 0 app.py
30-03-2024  17:07    <DIR>          artifacts
29-03-2024  17:12    <DIR>          config
29-03-2024  17:14                 0 Dockerfile
30-03-2024  07:47    <DIR>          env
29-03-2024  16:09             1,088 LICENSE
30-03-2024  08:08    <DIR>          logs
30-03-2024  08:07                91 main.py
30-03-2024  11:54                10 params.yaml
30-03-2024  10:56               299 README.md
29-03-2024  17:24               252 requirements.txt
30-03-2024  11:01    <DIR>          research
30-03-2024  07:40               807 setup.py
30-03-2024  07:56    <DIR>          src
29-03-2024  

In [43]:
%cd "./Text-Summarizer-project"

[WinError 2] The system cannot find the file specified: './Text-Summarizer-project'
c:\Users\Lenovo\Desktop\NLP PROJECTS\Text-Summarizer-project


c:\Users\Lenovo\Desktop\NLP PROJECTS\Text-Summarizer-project\env\lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [44]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\NLP PROJECTS\\Text-Summarizer-project'

entity

In [45]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path


In [46]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [47]:


class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
                
            self.config = read_yaml(config_file_path)
            self.config_file_path = read_yaml(params_file_path)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])
          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config     
                           

creating the components

In [48]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [53]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [55]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()


except Exception as e:
    raise e

[2024-03-30 17:13:49,987: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-30 17:13:49,989: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-30 17:13:49,991: INFO: common: created directory at: artifacts]
[2024-03-30 17:13:49,993: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-30 17:13:52,955: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8460:211DE0:4EA40F:6731CC:6607F6E1
Accept-Ranges: bytes
Date: Sat, 30 Mar 2024 11:43:50 GMT
Via: 1.1 varnish
X-Served-By: cache-del21738-DEL
X-Cache: H